## **Installation . . .**

In [1]:
import torch
if not torch.cuda.is_available():
    raise Exception("You should enable GPU runtime")


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


## **Installing tensorboard and setting it up . . .**

In this session, I wanted to use the original Tensorboard instead of using the TensorboardColab version. Doing this, for example, we are able to add images or graphs and not just scalars. Besides, we are able to load different experiments on the same graphics thus allowing us to compare them in the same plot.

In [3]:
# !pip install --upgrade tensorboard


In [4]:
%load_ext tensorboard


In [5]:
import os
logs_base_dir = "runs"
os.makedirs(logs_base_dir, exist_ok=True)


In [6]:
from torch.utils.tensorboard import SummaryWriter

tb_fm = SummaryWriter(log_dir=f'{logs_base_dir}/{logs_base_dir}_FM/')
tb_gcn = SummaryWriter(log_dir=f'{logs_base_dir}/{logs_base_dir}_GCN/')
tb_gcn_attention = SummaryWriter(
    log_dir=f'{logs_base_dir}/{logs_base_dir}_GCN_att/')


### Preparing imports

In [7]:
from torch.utils.data import DataLoader, Dataset
from IPython import embed
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np
import csv
import os
import scipy.sparse as sp
from tqdm import tqdm, trange


### **Downloading data and loading it with pandas ...**

In [8]:
# LOAD TRAINING DATA
data = pd.read_csv('./data/interactions.csv')
data


customer_id  article_id   timestamp     price  sales_channel_id
0             10054        5498  1590278400  0.033881                 2
1              8612        4092  1599177600  0.013542                 2
2             12973        4109  1591401600  0.013542                 1
3              9820       12918  1593820800  0.045763                 2
4              3329        1739  1578355200  0.011847                 2
...             ...         ...         ...       ...               ...
382317        12873       18345  1587772800  0.013542                 2
382318         1010       11221  1590710400  0.018390                 1
382319         8493       13473  1592784000  0.027102                 2
382320         7232        4383  1592179200  0.008458                 2
382321         5460       13815  1584057600  0.033881                 2

[382322 rows x 5 columns]

In [9]:
# Unique value for the label is 1 (we will need to manually sample negative data)
data.nunique()


customer_id         13359
article_id          21475
timestamp             265
price                4529
sales_channel_id        2
dtype: int64

In [10]:
data.shape


(382322, 5)

In [11]:
data = data.sort_values(by=['customer_id', 'timestamp'], ascending=[
                        True, False], ignore_index=True)
data


customer_id  article_id   timestamp     price  sales_channel_id
0                 0       21034  1598745600  0.016932                 2
1                 0         731  1598745600  0.013542                 2
2                 0        1219  1598745600  0.042356                 2
3                 0       20262  1597622400  0.033881                 2
4                 0        4910  1597622400  0.050831                 2
...             ...         ...         ...       ...               ...
382317        13358        5564  1581897600  0.033881                 2
382318        13358        9614  1581897600  0.016932                 2
382319        13358        4290  1581897600  0.025407                 2
382320        13358         326  1581724800  0.028407                 2
382321        13358         320  1578528000  0.033881                 2

[382322 rows x 5 columns]

In [12]:
data.drop_duplicates(
    subset=['customer_id', 'article_id'], keep='first', inplace=True)
data


customer_id  article_id   timestamp     price  sales_channel_id
0                 0       21034  1598745600  0.016932                 2
1                 0         731  1598745600  0.013542                 2
2                 0        1219  1598745600  0.042356                 2
3                 0       20262  1597622400  0.033881                 2
4                 0        4910  1597622400  0.050831                 2
...             ...         ...         ...       ...               ...
382317        13358        5564  1581897600  0.033881                 2
382318        13358        9614  1581897600  0.016932                 2
382319        13358        4290  1581897600  0.025407                 2
382320        13358         326  1581724800  0.028407                 2
382321        13358         320  1578528000  0.033881                 2

[351760 rows x 5 columns]

In [13]:
data = data[['customer_id', 'article_id']].copy()
data['label'] = 1
data


customer_id  article_id  label
0                 0       21034      1
1                 0         731      1
2                 0        1219      1
3                 0       20262      1
4                 0        4910      1
...             ...         ...    ...
382317        13358        5564      1
382318        13358        9614      1
382319        13358        4290      1
382320        13358         326      1
382321        13358         320      1

[351760 rows x 3 columns]

So, while we can observe for the training data many interactions for each user, we see below that for the testing (or validation) set we have just holded out one interaction for user, which will be used as ground-truth when evaluating the model outputing a ranking.

In [14]:
data['idx'] = data.index
data


customer_id  article_id  label     idx
0                 0       21034      1       0
1                 0         731      1       1
2                 0        1219      1       2
3                 0       20262      1       3
4                 0        4910      1       4
...             ...         ...    ...     ...
382317        13358        5564      1  382317
382318        13358        9614      1  382318
382319        13358        4290      1  382319
382320        13358         326      1  382320
382321        13358         320      1  382321

[351760 rows x 4 columns]

In [15]:
test_data = data.groupby('customer_id', as_index=False).first().set_index('idx')
test_data.index.name = None
test_data
e = test_data.copy()
e


customer_id  article_id  label
0                 0       21034      1
22                1        5111      1
85                2       18100      1
107               3       16343      1
141               4        2267      1
...             ...         ...    ...
382104        13354       20149      1
382130        13355       19574      1
382224        13356        2760      1
382272        13357       11818      1
382292        13358       16598      1

[13359 rows x 3 columns]

In [16]:
row_to_drop = test_data.index
test_data


customer_id  article_id  label
0                 0       21034      1
22                1        5111      1
85                2       18100      1
107               3       16343      1
141               4        2267      1
...             ...         ...    ...
382104        13354       20149      1
382130        13355       19574      1
382224        13356        2760      1
382272        13357       11818      1
382292        13358       16598      1

[13359 rows x 3 columns]

In [17]:
data.drop(index=row_to_drop.values, columns=['idx'], inplace=True)
d = data.copy()
data


customer_id  article_id  label
1                 0         731      1
2                 0        1219      1
3                 0       20262      1
4                 0        4910      1
5                 0       19321      1
...             ...         ...    ...
382317        13358        5564      1
382318        13358        9614      1
382319        13358        4290      1
382320        13358         326      1
382321        13358         320      1

[338401 rows x 3 columns]

In [18]:
assert 351760 - 338401 == 13359


### **Preprocessing dataset ...**


We will first show how to preprocess data for some individual examples in `1. Understanding how to process data` section and finally we will construct a *Pytorch Dataset class* which will allow us to preprocess and handle the whole data in order to forward it to the model (it is done in `2. Building dataset and preparing data for the model` section).


#### **1. Understanding how to process data...**

In [19]:
# userId,movieId,rating,timestamp
data = data.to_numpy()
data


array([[    0,   731,     1],
       [    0,  1219,     1],
       [    0, 20262,     1],
       ...,
       [13358,  4290,     1],
       [13358,   326,     1],
       [13358,   320,     1]])

In [20]:
items = data[:, :2].astype(int)
items


array([[    0,   731],
       [    0,  1219],
       [    0, 20262],
       ...,
       [13358,  4290],
       [13358,   326],
       [13358,   320]])

In [21]:
np.max(items, axis=0)[:2] + 1


array([13359, 21475])

In [22]:
# We need each node to have a unique id
reindex_items = items.copy()
reindex_items[:, 1] = reindex_items[:, 1] + 13359
reindex_items


array([[    0, 14090],
       [    0, 14578],
       [    0, 33621],
       ...,
       [13358, 17649],
       [13358, 13685],
       [13358, 13679]])

In [23]:
field_dims = np.max(reindex_items, axis=0) + 1
field_dims


array([13359, 34834])

In [24]:
def build_adj_mx(dims, interactions):
    train_mat = sp.dok_matrix((dims, dims), dtype=np.float32)
    for x in tqdm(interactions, desc="BUILDING ADJACENCY MATRIX..."):
        train_mat[x[0], x[1]] = 1.0
        train_mat[x[1], x[0]] = 1.0

    return train_mat


In [25]:
train_mat = build_adj_mx(field_dims[-1], reindex_items.copy())
train_mat


BUILDING ADJACENCY MATRIX...: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 338401/338401 [00:02<00:00, 135888.45it/s]


<34834x34834 sparse matrix of type '<class 'numpy.float32'>'
	with 676802 stored elements in Dictionary Of Keys format>

In [26]:
# Check that we have (2*99057 = 198114) interactions...
338401*2


676802

##### *Checking we have just positive data:*

In [27]:
targets = data[:, 2]
targets


array([1, 1, 1, ..., 1, 1, 1])

In [28]:
np.unique(targets)


array([1])

##### *Example on performing negative data for a training sample: (u, i, j)*

In [29]:
data = np.c_[(reindex_items, targets)].astype(int)
data


array([[    0, 14090,     1],
       [    0, 14578,     1],
       [    0, 33621,     1],
       ...,
       [13358, 17649,     1],
       [13358, 13685,     1],
       [13358, 13679,     1]])

In [30]:
field_dims[:2]


array([13359, 34834])

In [31]:
# EXAMPLE interaction number 988 : user 6 - item 1470
x = data[988]
x


array([   38, 15069,     1])

In [32]:
neg_triplet = np.array([0, 0, 0])
neg_triplet[0] = x[0].copy()
neg_triplet


array([38,  0,  0])

In [33]:
# Example: We find item 1200 has no connection with user 6
j = 1200
neg_triplet[1] = j
neg_triplet


array([  38, 1200,    0])

##### *Define metrics:*

In [34]:
import math


def getHitRatio(recommend_list, gt_item):
    if gt_item in recommend_list:
        return 1
    else:
        return 0


def getNDCG(recommend_list, gt_item):
    idx = np.where(recommend_list == gt_item)[0]
    if len(idx) > 0:
        return math.log(2)/math.log(idx+2)
    else:
        return 0


##### *Build test dataset for evaluation*

In [35]:
test_data


customer_id  article_id  label
0                 0       21034      1
22                1        5111      1
85                2       18100      1
107               3       16343      1
141               4        2267      1
...             ...         ...    ...
382104        13354       20149      1
382130        13355       19574      1
382224        13356        2760      1
382272        13357       11818      1
382292        13358       16598      1

[13359 rows x 3 columns]

In [36]:
test_data = test_data.to_numpy()
test_data.shape[0]


13359

In [37]:
# Take number of users and items from reindex items from train set
users, items = np.max(reindex_items, axis=0)[:2] + 1  # [ 943, 1682])
print(users)
print(items)


13359
34834


In [38]:
# Reindex test items and substract 1
pairs_test = test_data[:, :2].astype(int)
pairs_test[:, 1] = pairs_test[:, 1] + users
pairs_test


array([[    0, 34393],
       [    1, 18470],
       [    2, 31459],
       ...,
       [13356, 16119],
       [13357, 25177],
       [13358, 29957]])

In [39]:
assert 1219 + 13359 == 14578


In [40]:
pair = pairs_test[0]
pair


array([    0, 34393])

In [41]:
# GENERATE TEST SET WITH NEGATIVE EXAMPLES TO EVALUATE
# number users (943), number items (2625)
max_users, max_items = field_dims[:2]
negatives = []
for t in range(10):
    j = np.random.randint(max_users, max_items)
    while (pair[0], j) in train_mat or j == pair[1]:
        j = np.random.randint(max_users, max_items)
    negatives.append(j)
negatives


[18041, 18426, 20387, 16195, 32674, 30529, 14990, 20034, 17701, 22653]

In [42]:
single_user_test_set = np.vstack([pair, ] * (len(negatives)+1))
single_user_test_set


array([[    0, 34393],
       [    0, 34393],
       [    0, 34393],
       [    0, 34393],
       [    0, 34393],
       [    0, 34393],
       [    0, 34393],
       [    0, 34393],
       [    0, 34393],
       [    0, 34393],
       [    0, 34393]])

In [43]:
single_user_test_set[:, 1][1:] = negatives
single_user_test_set


array([[    0, 34393],
       [    0, 18041],
       [    0, 18426],
       [    0, 20387],
       [    0, 16195],
       [    0, 32674],
       [    0, 30529],
       [    0, 14990],
       [    0, 20034],
       [    0, 17701],
       [    0, 22653]])

#### **2. Building dataset and preparing data for the model ...**

In [44]:
d.to_numpy()[:, 2]


array([1, 1, 1, ..., 1, 1, 1])

In [45]:
e


customer_id  article_id  label
0                 0       21034      1
22                1        5111      1
85                2       18100      1
107               3       16343      1
141               4        2267      1
...             ...         ...    ...
382104        13354       20149      1
382130        13355       19574      1
382224        13356        2760      1
382272        13357       11818      1
382292        13358       16598      1

[13359 rows x 3 columns]

In [46]:
# @title
import numpy as np
import pandas as pd
import torch.utils.data


class HandMDataset(torch.utils.data.Dataset):
    """
    MovieLens 100k Dataset

    Data preparation
        treat samples with a rating less than 3 as negative samples

    :param dataset_path: MovieLens dataset path

    """

    def __init__(self, train_data, test_data, num_negatives_train=4, num_negatives_test=100):

        colnames = ["user_id", 'item_id', 'label']
        data = train_data.to_numpy()
        test_data = test_data.to_numpy()

        # TAKE items, targets and test_items
        self.targets = data[:, 2]
        self.items = self.preprocess_items(data)

        # Save dimensions of max users and items and build training matrix
        self.field_dims = np.max(self.items, axis=0) + 1  # ([ 943, 2625])
        print(self.field_dims)
        self.train_mat = self.__build_adj_mx__(self.field_dims[-1], self.items.copy())

        # Generate train interactions with 4 negative samples for each positive
        self.negative_sampling(num_negatives=num_negatives_train)

        # Build test set by passing as input the test item interactions
        self.test_set = self.build_test_set(self.preprocess_items(test_data), num_neg_samples_test=num_negatives_test)

    def __len__(self):
        return self.targets.shape[0]

    def __getitem__(self, index):
        return self.interactions[index]

    def preprocess_items(self, data, users=13359):
        reindexed_items = data[:, :2].astype(
            np.int)  # -1 because ID begins from 1
        # users, items = np.max(reindexed_items, axis=0)[:2] + 1 # [ 943, 1682])
        # Reindex items (we need to have [users + items] nodes with unique idx)
        reindexed_items[:, 1] = reindexed_items[:, 1] + users

        return reindexed_items
    
    def __build_adj_mx__(self, dims, interactions):
        train_mat = sp.dok_matrix((dims, dims), dtype=np.float32)
        for x in tqdm(interactions, desc="BUILDING ADJACENCY MATRIX..."):
            train_mat[x[0], x[1]] = 1.0
            train_mat[x[1], x[0]] = 1.0

        return train_mat

    def negative_sampling(self, num_negatives=4):
        self.interactions = []
        data = np.c_[(self.items, self.targets)].astype(int)
        # number users (943), number items (2625)
        max_users, max_items = self.field_dims[:2]

        # x are triplets (u, i , 1)
        for x in tqdm(data, desc="Performing negative sampling on test data..."):
            # Append positive interaction
            self.interactions.append(x)
            # Copy user and maintain last position to 0. Now we will need to update neg_triplet[1] with j
            neg_triplet = np.vstack([x, ] * (num_negatives))
            neg_triplet[:, 2] = np.zeros(num_negatives)

            # Generate num_negatives negative interactions
            for idx in range(num_negatives):
                j = np.random.randint(max_users, max_items)
                # IDEA: Loop to exclude true interactions (set to 1 in adj_train) user - item
                while (x[0], j) in self.train_mat:
                    j = np.random.randint(max_users, max_items)
                neg_triplet[:, 1][idx] = j
            self.interactions.append(neg_triplet.copy())

        self.interactions = np.vstack(self.interactions)

    def build_test_set(self, gt_test_interactions, num_neg_samples_test=99):
        # number users (943), number items (2625)
        max_users, max_items = self.field_dims[:2]
        test_set = []
        for pair in tqdm(gt_test_interactions, desc="BUILDING TEST SET..."):
            negatives = []
            for t in range(num_neg_samples_test):
                j = np.random.randint(max_users, max_items)
                while (pair[0], j) in self.train_mat or j == pair[1]:
                    j = np.random.randint(max_users, max_items)
                negatives.append(j)
            # APPEND TEST SETS FOR SINGLE USER
            single_user_test_set = np.vstack([pair, ] * (len(negatives)+1))
            single_user_test_set[:, 1][1:] = negatives
            test_set.append(single_user_test_set.copy())
        return test_set


In [47]:
full_dataset = HandMDataset(d, e, num_negatives_train=4, num_negatives_test=99)

/tmp/ipykernel_64042/3924630987.py:47: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.int)  # -1 because ID begins from 1


[13359 34834]


BUILDING TEST SET...: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13359/13359 [00:02<00:00, 6445.09it/s]


In [48]:
# 90570 interactions with pairs of index that have interacted + 4*90570 negative
full_dataset.interactions

array([[    0, 14090,     1],
       [    0, 24899,     0],
       [    0, 32610,     0],
       ...,
       [13358, 14023,     0],
       [13358, 18762,     0],
       [13358, 28192,     0]])

In [49]:
full_dataset.interactions[:20]

array([[    0, 14090,     1],
       [    0, 24899,     0],
       [    0, 32610,     0],
       [    0, 30558,     0],
       [    0, 19840,     0],
       [    0, 14578,     1],
       [    0, 21141,     0],
       [    0, 25411,     0],
       [    0, 33792,     0],
       [    0, 18114,     0],
       [    0, 33621,     1],
       [    0, 24271,     0],
       [    0, 17755,     0],
       [    0, 20814,     0],
       [    0, 29444,     0],
       [    0, 18269,     1],
       [    0, 15268,     0],
       [    0, 27664,     0],
       [    0, 24805,     0],
       [    0, 22362,     0]])

In [50]:
# We had 99057 interactions in training_matrix --> now we have 99057 positive plus 4*99057 negative
assert 5*338401 == full_dataset.interactions.shape[0]

In [51]:
# For test set, we keep the size (one interaction per user) but we append 99 negative samples for evaluation
print(len(full_dataset.test_set))


13359


In [52]:
len(full_dataset.test_set[0])  # --> [gt_pair + 99_neg_samples]


100

In [53]:
full_dataset.test_set[0]


array([[    0, 34393],
       [    0, 26636],
       [    0, 29864],
       [    0, 28594],
       [    0, 22753],
       [    0, 31316],
       [    0, 20281],
       [    0, 20554],
       [    0, 24192],
       [    0, 18523],
       [    0, 17137],
       [    0, 32634],
       [    0, 21679],
       [    0, 20019],
       [    0, 28378],
       [    0, 18327],
       [    0, 19640],
       [    0, 27428],
       [    0, 22973],
       [    0, 23632],
       [    0, 23188],
       [    0, 16360],
       [    0, 33068],
       [    0, 13395],
       [    0, 25698],
       [    0, 15001],
       [    0, 24086],
       [    0, 32803],
       [    0, 34379],
       [    0, 23234],
       [    0, 18898],
       [    0, 24768],
       [    0, 23801],
       [    0, 17294],
       [    0, 30999],
       [    0, 32042],
       [    0, 22017],
       [    0, 17102],
       [    0, 24507],
       [    0, 14311],
       [    0, 31748],
       [    0, 34683],
       [    0, 29610],
       [   

Sampling 4 negative samples for each positive, will also work as a type of normalization.

In [54]:
data_loader = DataLoader(full_dataset, batch_size=256, shuffle=True, num_workers=0)

In [55]:
for i, (interactions) in enumerate(data_loader):
    if i == 0:
        print(interactions.shape)
    else:
        break


torch.Size([256, 3])


### **Building Factorization Machines model**


Our training matrix is now even sparser: Of all 237,746,250 values (90,570*2,625), only 181,140 are non-zero (90,570*2). In other words, the matrix is 99.92% sparse. Storing this as a dense matrix would be a massive waste of both storage and computing power!
To avoid this, let’s use a scipy.lil_matrix sparse matrix for samples and a numpy array for labels.



<div>
<center><img src="https://d2908q01vomqb2.cloudfront.net/f1f836cb4ea6efb2a0b1b99f41ad8b103eff4b59/2019/04/03/sagemaker-factorization-1.gif" width="400"/></center>
</div>

##### **LAYERS:** Linear and FM part of the equation

In [56]:
# EMBEDDING PYTORCH: https://pytorch.org/docs/stable/nn.html?highlight=embedding#torch.nn.Embedding


In [57]:
# Linear part of the equation
class FeaturesLinear(torch.nn.Module):

    def __init__(self, field_dims, output_dim=1):
        super().__init__()

        self.fc = torch.nn.Embedding(field_dims, output_dim)
        self.bias = torch.nn.Parameter(torch.zeros((output_dim,)))

    def forward(self, x):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        # self.fc(x).shape --> [batch_size, num_fields, 1]
        # torch.sum(self.fc(x), dim=1).shape --> ([batch_size, 1])
        return torch.sum(self.fc(x), dim=1) + self.bias


In [58]:
# FM part of the equation
class FM_operation(torch.nn.Module):

    def __init__(self, reduce_sum=True):
        super().__init__()
        self.reduce_sum = reduce_sum

    def forward(self, x):
        """
        :param x: Float tensor of size ``(batch_size, num_fields, embed_dim)``
        """
        square_of_sum = torch.sum(x, dim=1) ** 2
        sum_of_square = torch.sum(x ** 2, dim=1)
        ix = square_of_sum - sum_of_square
        if self.reduce_sum:
            ix = torch.sum(ix, dim=1, keepdim=True)
        return 0.5 * ix


##### MODEL

In [59]:
from torch_geometric.nn import GCNConv


class FactorizationMachineModel(torch.nn.Module):
    """
    A pytorch implementation of Factorization Machine.

    Reference:
        S Rendle, Factorization Machines, 2010.
    """

    def __init__(self, field_dims, embed_dim):
        super().__init__()
        # field_dims == total of nodes (sum users + context)
        # self.linear = torch.nn.Linear(field_dims, 1, bias=True)
        self.linear = FeaturesLinear(field_dims)
        self.embedding = torch.nn.Embedding(
            field_dims, embed_dim, sparse=False)
        self.fm = FM_operation(reduce_sum=True)

        torch.nn.init.xavier_uniform_(self.embedding.weight.data)

    def forward(self, interaction_pairs):
        """
        :param interaction_pairs: Long tensor of size ``(batch_size, num_fields)``
        """
        out = self.linear(interaction_pairs) + \
            self.fm(self.embedding(interaction_pairs))

        return out.squeeze(1)

    def predict(self, interactions, device):
        # return the score, inputs are numpy arrays, outputs are tensors
        test_interactions = torch.from_numpy(
            interactions).to(dtype=torch.long, device=device)
        output_scores = self.forward(test_interactions)
        return output_scores


### **Workflow for FM with usual embeddings ...**

#### **Train**

In [60]:
from statistics import mean


def train_one_epoch(model, optimizer, data_loader, criterion, device, log_interval=100):
    model.train()
    total_loss = []

    for i, (interactions) in enumerate(data_loader):
        interactions = interactions.to(device)
        targets = interactions[:, 2]
        predictions = model(interactions[:, :2])

        loss = criterion(predictions, targets.float())
        model.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss.append(loss.item())

    return mean(total_loss)


#### **Evaluation**

##### **Understanding evaluation ...**

In [61]:
len(full_dataset.test_set)


13359

In [62]:
user_test = full_dataset.test_set[0]
user_test.shape


(100, 2)

In [63]:
user_test


array([[    0, 34393],
       [    0, 26636],
       [    0, 29864],
       [    0, 28594],
       [    0, 22753],
       [    0, 31316],
       [    0, 20281],
       [    0, 20554],
       [    0, 24192],
       [    0, 18523],
       [    0, 17137],
       [    0, 32634],
       [    0, 21679],
       [    0, 20019],
       [    0, 28378],
       [    0, 18327],
       [    0, 19640],
       [    0, 27428],
       [    0, 22973],
       [    0, 23632],
       [    0, 23188],
       [    0, 16360],
       [    0, 33068],
       [    0, 13395],
       [    0, 25698],
       [    0, 15001],
       [    0, 24086],
       [    0, 32803],
       [    0, 34379],
       [    0, 23234],
       [    0, 18898],
       [    0, 24768],
       [    0, 23801],
       [    0, 17294],
       [    0, 30999],
       [    0, 32042],
       [    0, 22017],
       [    0, 17102],
       [    0, 24507],
       [    0, 14311],
       [    0, 31748],
       [    0, 34683],
       [    0, 29610],
       [   

In [64]:
gt_pair = user_test[0]
neg_items = user_test[1:]
print(f'gt_pair: {gt_pair}')
print(f'lenght neg_items: {len(neg_items)}')


gt_pair: [    0 34393]
lenght neg_items: 99


In [65]:
# DEFINE GT_ITEM
gt_item = user_test[0][1]
gt_item


34393

In [66]:
# Defining dummy model with 8 embedding dimensions
dummy_model = FactorizationMachineModel(
    full_dataset.field_dims[-1], 8).to(device)
out = dummy_model.predict(user_test, device)
out.shape


torch.Size([100])

In [67]:
# Print first 10 predictions, where 1st one is the one for the GT
out[:10]


tensor([-1.8204, -3.0537, -0.7185, -0.9135, -0.2395, -0.8642,  0.3299, -1.5350,
        -0.1850,  0.7495], device='cuda:0', grad_fn=<SliceBackward0>)

In [68]:
values, indices = torch.topk(out, 10)
print(values)
print(indices.cpu().detach().numpy())


tensor([1.6432, 0.9790, 0.8414, 0.7495, 0.6781, 0.6704, 0.5596, 0.5246, 0.5011,
        0.5010], device='cuda:0', grad_fn=<TopkBackward0>)
[16 65 27  9 97 39 56 93 11 33]


In [69]:
user_test[0]


array([    0, 34393])

In [70]:
# RANKING LIST TO RECOMMEND
recommend_list = user_test[indices.cpu().detach().numpy()][:, 1]
recommend_list


array([19640, 30524, 32803, 18523, 31118, 14311, 20308, 30411, 32634,
       17294])

In [71]:
gt_item in recommend_list


False

##### **Defining test function...**

In [72]:
def test(model, full_dataset, device, topk=10):
    # Test the HR and NDCG for the model @topK
    model.eval()

    HR, NDCG = [], []

    for user_test in full_dataset.test_set:
        gt_item = user_test[0][1]

        predictions = model.predict(user_test, device)
        _, indices = torch.topk(predictions, topk)
        recommend_list = user_test[indices.cpu().detach().numpy()][:, 1]

        HR.append(getHitRatio(recommend_list, gt_item))
        NDCG.append(getNDCG(recommend_list, gt_item))
    return mean(HR), mean(NDCG)


#### **Model, loss and optimizer definition**

In [73]:
model = FactorizationMachineModel(full_dataset.field_dims[-1], 32).to(device)


In [74]:
criterion = torch.nn.BCEWithLogitsLoss(reduction='mean')
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)


#### **Random evaluation**

In [75]:
topk = 10

# Check Init performance
hr, ndcg = test(model, full_dataset, device, topk=topk)
print("initial HR: ", hr)
print("initial NDCG: ", ndcg)


initial HR:  0.10397484841679766
initial NDCG:  0.04787105153241264


#### **Start training the model**

In [76]:
# DO EPOCHS NOW
tb = True
topk = 10
for epoch_i in range(20):
    # data_loader.dataset.negative_sampling()
    train_loss = train_one_epoch(
        model, optimizer, data_loader, criterion, device)
    hr, ndcg = test(model, full_dataset, device, topk=topk)

    print('\n')

    print(f'epoch {epoch_i}:')
    print(
        f'training loss = {train_loss:.4f} | Eval: HR@{topk} = {hr:.4f}, NDCG@{topk} = {ndcg:.4f} ')
    print('\n')
    if tb:
        tb_fm.add_scalar('train/loss', train_loss, epoch_i)
        tb_fm.add_scalar('eval/HR@{topk}', hr, epoch_i)
        tb_fm.add_scalar('eval/NDCG@{topk}', ndcg, epoch_i)




epoch 0:
training loss = 0.7504 | Eval: HR@10 = 0.1082, NDCG@10 = 0.0498 




epoch 1:
training loss = 0.5703 | Eval: HR@10 = 0.1246, NDCG@10 = 0.0589 




epoch 2:
training loss = 0.4628 | Eval: HR@10 = 0.1404, NDCG@10 = 0.0679 




epoch 3:
training loss = 0.3986 | Eval: HR@10 = 0.1473, NDCG@10 = 0.0711 




epoch 4:
training loss = 0.3491 | Eval: HR@10 = 0.1484, NDCG@10 = 0.0718 




epoch 5:
training loss = 0.3016 | Eval: HR@10 = 0.1460, NDCG@10 = 0.0710 




epoch 6:
training loss = 0.2556 | Eval: HR@10 = 0.1451, NDCG@10 = 0.0706 




epoch 7:
training loss = 0.2127 | Eval: HR@10 = 0.1449, NDCG@10 = 0.0705 




epoch 8:
training loss = 0.1747 | Eval: HR@10 = 0.1451, NDCG@10 = 0.0703 




epoch 9:
training loss = 0.1421 | Eval: HR@10 = 0.1446, NDCG@10 = 0.0700 




epoch 10:
training loss = 0.1147 | Eval: HR@10 = 0.1467, NDCG@10 = 0.0704 




epoch 11:
training loss = 0.0919 | Eval: HR@10 = 0.1481, NDCG@10 = 0.0705 




epoch 12:
training loss = 0.0732 | Eval: HR@10 = 0.1493, NDC

## **VISUALIZING RESULTS**

Once we have trained both models (*fm with usual embbedding layers* vs *fm with embeddings from gcn*), we can observe both metrics and loss in the same graphic in order to compare:

In [77]:
%tensorboard --logdir runs

# OUR MODEL 

In [78]:
from torch_geometric.nn import GCNConv
from scipy.sparse import identity
from torch_geometric.utils import dense_to_sparse, from_scipy_sparse_matrix


class GCNModel(torch.nn.Module):
    def __init__(self, field_dims, embed_dim, A):
        super().__init__()
        self.features = torch.Tensor(np.identity(A.shape[0])).to(device)
        self.gcn = GCNConv(field_dims, embed_dim)
        self.A = A
        self.edge_index_t, self.edge_weights = from_scipy_sparse_matrix(self.A)

        self.edge_index_t = self.edge_index_t.to(device)

    def forward(self, x):
        # x, edge_index = x.x, x.edge_index
        gcn = self.gcn(self.features, self.edge_index_t)[x]
        return gcn


In [79]:
class FMGCNModel(torch.nn.Module):
    def __init__(self, field_dims, embed_dim, A):
        super().__init__()
        # field_dims == total of nodes (sum users + context)
        # self.linear = torch.nn.Linear(field_dims, 1, bias=True)
        self.linear = FeaturesLinear(field_dims)
        # self.embedding = torch.nn.Embedding(field_dims, embed_dim, sparse=False)
        self.embedding = GCNModel(field_dims, embed_dim, A)
        self.fm = FM_operation(reduce_sum=True)

        # torch.nn.init.xavier_uniform_(self.embedding.)

    def forward(self, interaction_pairs):
        """
        :param interaction_pairs: Long tensor of size ``(batch_size, num_fields)``
        """
        out = self.linear(interaction_pairs) + \
            self.fm(self.embedding(interaction_pairs))

        return out.squeeze(1)

    def predict(self, interactions, device):
        # return the score, inputs are numpy arrays, outputs are tensors
        test_interactions = torch.from_numpy(
            interactions).to(dtype=torch.long, device=device)
        output_scores = self.forward(test_interactions)
        return output_scores


In [80]:
# Defining dummy model with 8 embedding dimensions
dummy_model = FMGCNModel(full_dataset.field_dims[-1], 8, full_dataset.train_mat).to(device)
out = dummy_model.predict(user_test, device)
out.shape


torch.Size([100])

In [81]:
model = FMGCNModel(full_dataset.field_dims[-1], 32, full_dataset.train_mat).to(device)


In [82]:
criterion = torch.nn.BCEWithLogitsLoss(reduction='mean')
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)


In [83]:
# DO EPOCHS NOW
tb = True
topk = 10
for epoch_i in range(20):
    # data_loader.dataset.negative_sampling()
    train_loss = train_one_epoch(
        model, optimizer, data_loader, criterion, device)
    hr, ndcg = test(model, full_dataset, device, topk=topk)

    print('\n')

    print(f'epoch {epoch_i}:')
    print(
        f'training loss = {train_loss:.4f} | Eval: HR@{topk} = {hr:.4f}, NDCG@{topk} = {ndcg:.4f} ')
    print('\n')
    if tb:
        tb_gcn.add_scalar('train/loss', train_loss, epoch_i)
        tb_gcn.add_scalar('eval/HR@{topk}', hr, epoch_i)
        tb_gcn.add_scalar('eval/NDCG@{topk}', ndcg, epoch_i)




epoch 0:
training loss = 0.6879 | Eval: HR@10 = 0.1096, NDCG@10 = 0.0518 




epoch 1:
training loss = 0.5313 | Eval: HR@10 = 0.1650, NDCG@10 = 0.0829 




epoch 2:
training loss = 0.4715 | Eval: HR@10 = 0.2023, NDCG@10 = 0.1039 




epoch 3:
training loss = 0.4390 | Eval: HR@10 = 0.2240, NDCG@10 = 0.1162 




epoch 4:
training loss = 0.4210 | Eval: HR@10 = 0.2309, NDCG@10 = 0.1200 




epoch 5:
training loss = 0.4092 | Eval: HR@10 = 0.2359, NDCG@10 = 0.1230 




epoch 6:
training loss = 0.3998 | Eval: HR@10 = 0.2381, NDCG@10 = 0.1238 




epoch 7:
training loss = 0.3910 | Eval: HR@10 = 0.2398, NDCG@10 = 0.1253 




epoch 8:
training loss = 0.3817 | Eval: HR@10 = 0.2412, NDCG@10 = 0.1261 




epoch 9:
training loss = 0.3711 | Eval: HR@10 = 0.2434, NDCG@10 = 0.1276 




epoch 10:
training loss = 0.3589 | Eval: HR@10 = 0.2476, NDCG@10 = 0.1300 




epoch 11:
training loss = 0.3452 | Eval: HR@10 = 0.2529, NDCG@10 = 0.1330 




epoch 12:
training loss = 0.3302 | Eval: HR@10 = 0.2577, NDC

In [84]:
from torch_geometric.nn import GATConv

class GCNAttModel(torch.nn.Module):
    def __init__(self, field_dims, embed_dim, A):
        super().__init__()
        self.features = torch.Tensor(np.identity(A.shape[0])).to(device)
        self.gcn = GATConv(int(field_dims), embed_dim)         
        self.A = A
        self.edge_index_t, self.edge_weights = from_scipy_sparse_matrix(self.A)

        self.edge_index_t = self.edge_index_t.to(device)

    def forward(self, x):
        # x, edge_index = x.x, x.edge_index
        gcn = self.gcn(self.features, self.edge_index_t)[x]
        return gcn
    

class FMGCNAttModel(torch.nn.Module):
    def __init__(self, field_dims, embed_dim, A):
        super().__init__()
        # field_dims == total of nodes (sum users + context)
        # self.linear = torch.nn.Linear(field_dims, 1, bias=True)
        self.linear = FeaturesLinear(field_dims)
        self.embedding = GCNAttModel(field_dims, embed_dim, A)
        # self.embedding = GCNModel(field_dims, embed_dim, A)
        self.fm = FM_operation(reduce_sum=True)

        # torch.nn.init.xavier_uniform_(self.embedding.)

    def forward(self, interaction_pairs):
        """
        :param interaction_pairs: Long tensor of size ``(batch_size, num_fields)``
        """
        out = self.linear(interaction_pairs) + \
            self.fm(self.embedding(interaction_pairs))

        return out.squeeze(1)

    def predict(self, interactions, device):
        # return the score, inputs are numpy arrays, outputs are tensors
        test_interactions = torch.from_numpy(
            interactions).to(dtype=torch.long, device=device)
        output_scores = self.forward(test_interactions)
        return output_scores


In [85]:
torch.cuda.empty_cache()
import gc
gc.collect()

32

In [86]:
type(full_dataset.field_dims[-1])


numpy.int64

In [87]:
# Defining dummy model with 8 embedding dimensions
dummy_model = FMGCNAttModel(full_dataset.field_dims[-1], 8, full_dataset.train_mat).to(device)
out = dummy_model.predict(user_test, device)
out.shape


torch.Size([100])

In [88]:
model = FMGCNAttModel(full_dataset.field_dims[-1], 32, full_dataset.train_mat).to(device)


In [89]:
criterion = torch.nn.BCEWithLogitsLoss(reduction='mean')
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)


In [90]:
# DO EPOCHS NOW
tb = True
topk = 10
for epoch_i in range(20):
    # data_loader.dataset.negative_sampling()
    train_loss = train_one_epoch(
        model, optimizer, data_loader, criterion, device)
    hr, ndcg = test(model, full_dataset, device, topk=topk)

    print('\n')

    print(f'epoch {epoch_i}:')
    print(
        f'training loss = {train_loss:.4f} | Eval: HR@{topk} = {hr:.4f}, NDCG@{topk} = {ndcg:.4f} ')
    print('\n')
    if tb:
        tb_gcn_attention.add_scalar('train/loss', train_loss, epoch_i)
        tb_gcn_attention.add_scalar('eval/HR@{topk}', hr, epoch_i)
        tb_gcn_attention.add_scalar('eval/NDCG@{topk}', ndcg, epoch_i)




epoch 0:
training loss = 0.6140 | Eval: HR@10 = 0.2309, NDCG@10 = 0.1206 




epoch 1:
training loss = 0.4887 | Eval: HR@10 = 0.2427, NDCG@10 = 0.1259 




epoch 2:
training loss = 0.4600 | Eval: HR@10 = 0.2518, NDCG@10 = 0.1301 




epoch 3:
training loss = 0.4413 | Eval: HR@10 = 0.2505, NDCG@10 = 0.1306 




epoch 4:
training loss = 0.4280 | Eval: HR@10 = 0.2546, NDCG@10 = 0.1321 




epoch 5:
training loss = 0.4181 | Eval: HR@10 = 0.2541, NDCG@10 = 0.1313 




epoch 6:
training loss = 0.4097 | Eval: HR@10 = 0.2538, NDCG@10 = 0.1304 




epoch 7:
training loss = 0.4016 | Eval: HR@10 = 0.2523, NDCG@10 = 0.1300 




epoch 8:
training loss = 0.3923 | Eval: HR@10 = 0.2513, NDCG@10 = 0.1291 




epoch 9:
training loss = 0.3804 | Eval: HR@10 = 0.2503, NDCG@10 = 0.1286 




epoch 10:
training loss = 0.3650 | Eval: HR@10 = 0.2462, NDCG@10 = 0.1262 




epoch 11:
training loss = 0.3456 | Eval: HR@10 = 0.2454, NDCG@10 = 0.1252 




epoch 12:
training loss = 0.3233 | Eval: HR@10 = 0.2422, NDC

In [91]:
%tensorboard --logdir runs

Reusing TensorBoard on port 6006 (pid 64230), started 1:25:09 ago. (Use '!kill 64230' to kill it.)